<a href="https://colab.research.google.com/github/Priya378/LR-1-_Parser/blob/main/LR(1)_Parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Grammar:**

In [ ]:
"""
S->V=E|E
E->V
V->x|*E
"""

### **Taking Context Free Grammar as input and augmenting**

In [ ]:
#Declaring empty lists
productionRules=[] #Production rules   
nonTerminals=[]# non terminals 
rhsAugmented=[]#of augmented rules 
lhsAugmented=[] #augmented rules
 
print("Enter production rules, input $ to terminate: ")
#Taking first rule as input and adding $ to terminate
a=input()
productionRules.append("G->"+a[0]+"$")
lhsAugmented.append("G")
nonTerminals.append("G")
rhsAugmented.append(a[0]+"$")
 
#Take rules until "$" is given as an input
while(a!="$"):
  productionRules.append(a)
  a=input()
 
#Generating lhs and rhs for the augmented grammar
for a in productionRules[1:]:
  nonTerminals.append(a.split("->")[0])
  lhsAugmented+=[a.split("->")[0] for m in a.split("->")[1].split("|") ]
  rhsAugmented+=a.split("->")[1].split("|")
 
#Generating rules for the augmented grammar
rules=[]
for i in range(len(lhsAugmented)):
  rules.append(lhsAugmented[i]+"->"+rhsAugmented[i])
 
#Generating terminals of the grammar
terminals=[]
for x in rhsAugmented:
  for char in x:
    if(char not in nonTerminals and char not in terminals):
      terminals.append(char)
 
#Printing rules
print("\nAugmented Grammar: ")
for i in range(len(rules)):
  print(i," ",rules[i])

Enter production rules, input $ to terminate: 
S->V=E|E
E->V
V->x|*E
$

Augmented Grammar: 
0   G->S$
1   S->V=E
2   S->E
3   E->V
4   V->x
5   V->*E


### **Defining modules to find**
*   First sets
*   Closure
*   Symbols in a state
*   Go to function







In [ ]:
def first(String):
  if(String[0] not in nonTerminals):
    return String[0]
  firstSet=[]
  for rule in rules:
    if(rule[0]==String[0]):
      firstInRHS=rule.split("->")[1][0]
      if(firstInRHS not in nonTerminals and firstInRHS not in firstSet):
        firstSet.append(firstInRHS)
      elif(firstInRHS!=String[0]):
        tempFirstSet=first(firstInRHS)
        firstSet+=[i for i in tempFirstSet if i not in firstSet]
  return firstSet
 
def closure(i,rule,lookahead):
  lst=[[i,rule,lookahead]]
  rhsOfRule=rule.split("->")[1] 
  #Confirming dot isn't in the end of the rule
  if(len(rhsOfRule)!=i):
    for lstRule in lst:
      rhs=lstRule[1].split("->")[1]
      for rule in rules:
        if(rule[0]==rhs[lstRule[0]]):
          lookaheads=first(rhs[i+1:]+lstRule[2])
          for la in lookaheads:
            temp=[0,rule,la]
            if(temp not in lst):
              lst.append(temp)
  return lst
 
def symbols(state):
  symb=[]
  for entry in state:
    rhs=entry[1].split("->")[1]
    #Confirming if dot hasn't reached the end of rule
    if(len(rhs)!=entry[0]):
      temp=rhs[entry[0]]
      if(temp not in symb):
        symb.append(temp)
  return symb
 
def goto(state,symbol):
  #Checking for the accept state
  if(state[0][2]=="?" and symbol=='$'):
    index=state[0][0]
    if(state[0][1].split("->")[1][index]=="$"):
      return "acc"
 
  newState=[]
  for rule in state:
    rhs=rule[1].split("->")[1]
    if(len(rhs)!=rule[0]):
      if(rhs[rule[0]]==symbol):
        stateForSymbol=closure(rule[0]+1,rule[1],rule[2])
        for tempRule in stateForSymbol:
          if(tempRule not in newState):
            newState.append(tempRule)
  return newState

In [ ]:
firstState=closure(0,rules[0],"?")
for i in firstState:
  print(i)

[0, 'G->S$', '?']
[0, 'S->V=E', '$']
[0, 'S->E', '$']
[0, 'V->x', '=']
[0, 'V->*E', '=']
[0, 'E->V', '$']
[0, 'V->x', '$']
[0, 'V->*E', '$']


### **Driver Code**


*   Creating all states
*   Making Entries in parsing table



In [ ]:
#-------------------------
#Creating a dictionary with set of terminals and non terminals as keys to use in parsing table
columns=terminals+nonTerminals
columns.remove("G")
emptyRow=dict.fromkeys(columns)
for i in emptyRow:
  emptyRow[i]="#"
table=[]
 
#Driver Code along with making entries in table
C=[closure(0,rules[0],"?")]
for state in C:
  #Make a copy of empty row for each state in C
  row=emptyRow.copy()
  symbolsList=symbols(state)
  #For all the symbols in a particular state
  for symbol in symbolsList:
   next=goto(state,symbol)
   if(next=="acc"):
     row[symbol]="acc"
   elif(len(next)!=0):
    if(next not in C):
      C.append(next)
    if(symbol not in nonTerminals):
        row[symbol]="S"+str(C.index(next))
    else:
        row[symbol]=str(C.index(next))
  table.append(row)
 
#Filling for the reductions
for state in C:
  for entry in state:
    rh=entry[1].split("->")[1]
    if(len(rh)==entry[0]):
      table[C.index(state)][entry[2]]="R"+str(rules.index(entry[1]))
#-------------------------------------------------

In [ ]:
print(C)

[[[0, 'G->S$', '?'], [0, 'S->V=E', '$'], [0, 'S->E', '$'], [0, 'V->x', '='], [0, 'V->*E', '='], [0, 'E->V', '$'], [0, 'V->x', '$'], [0, 'V->*E', '$']], [[1, 'G->S$', '?']], [[1, 'S->V=E', '$'], [1, 'E->V', '$']], [[1, 'S->E', '$']], [[1, 'V->x', '='], [1, 'V->x', '$']], [[1, 'V->*E', '='], [0, 'E->V', '='], [0, 'V->x', '='], [0, 'V->*E', '='], [1, 'V->*E', '$'], [0, 'E->V', '$'], [0, 'V->x', '$'], [0, 'V->*E', '$']], [[2, 'S->V=E', '$'], [0, 'E->V', '$'], [0, 'V->x', '$'], [0, 'V->*E', '$']], [[2, 'V->*E', '='], [2, 'V->*E', '$']], [[1, 'E->V', '='], [1, 'E->V', '$']], [[3, 'S->V=E', '$']], [[1, 'E->V', '$']], [[1, 'V->x', '$']], [[1, 'V->*E', '$'], [0, 'E->V', '$'], [0, 'V->x', '$'], [0, 'V->*E', '$']], [[2, 'V->*E', '$']]]


### **Printing States**

In [ ]:
#Printing states:
from prettytable import PrettyTable
print("\n\nStates: ")
print("LS stands for look ahead symbols")
for i in range(len(C)):
  print("\nI"+str(i)+":")
  State=PrettyTable(["Rule", "LS"])
  for j in C[i]:
    dotposition=j[0]
    lhs=j[1].split("->")[0]
    rhs=j[1].split("->")[1]
    rhswithdot=rhs[0:dotposition]+"."+rhs[dotposition:]
    rule=lhs+"->"+rhswithdot
    State.add_row([rule,j[2]])
  print(State)



States: 
LS stands for look ahead symbols

I0:
+---------+----+
|   Rule  | LS |
+---------+----+
|  G->.S$ | ?  |
| S->.V=E | $  |
|  S->.E  | $  |
|  V->.x  | =  |
|  V->.*E | =  |
|  E->.V  | $  |
|  V->.x  | $  |
|  V->.*E | $  |
+---------+----+

I1:
+--------+----+
|  Rule  | LS |
+--------+----+
| G->S.$ | ?  |
+--------+----+

I2:
+---------+----+
|   Rule  | LS |
+---------+----+
| S->V.=E | $  |
|  E->V.  | $  |
+---------+----+

I3:
+-------+----+
|  Rule | LS |
+-------+----+
| S->E. | $  |
+-------+----+

I4:
+-------+----+
|  Rule | LS |
+-------+----+
| V->x. | =  |
| V->x. | $  |
+-------+----+

I5:
+--------+----+
|  Rule  | LS |
+--------+----+
| V->*.E | =  |
| E->.V  | =  |
| V->.x  | =  |
| V->.*E | =  |
| V->*.E | $  |
| E->.V  | $  |
| V->.x  | $  |
| V->.*E | $  |
+--------+----+

I6:
+---------+----+
|   Rule  | LS |
+---------+----+
| S->V=.E | $  |
|  E->.V  | $  |
|  V->.x  | $  |
|  V->.*E | $  |
+---------+----+

I7:
+--------+----+
|  Rule  | LS |
+----

### **Printing Parsing Table**

In [ ]:
#Printing parsing table
print("Parsing Table:\n")
print("\t","\t".join(columns))
for i in range(len(table)):
  print(i, end="\t")
  for j in table[i]:
      print(table[i][j],end="\t")
  print()

Parsing Table:

	 $	=	x	*	S	E	V
0	#	#	S4	S5	1	3	2	
1	acc	#	#	#	#	#	#	
2	R3	S6	#	#	#	#	#	
3	R2	#	#	#	#	#	#	
4	R4	R4	#	#	#	#	#	
5	#	#	S4	S5	#	7	8	
6	#	#	S11	S12	#	9	10	
7	R5	R5	#	#	#	#	#	
8	R3	R3	#	#	#	#	#	
9	R1	#	#	#	#	#	#	
10	R3	#	#	#	#	#	#	
11	R4	#	#	#	#	#	#	
12	#	#	S11	S12	#	13	10	
13	R5	#	#	#	#	#	#	


**Parsing String**

In [ ]:
from prettytable import PrettyTable
inp=input("Input the string to be parsed: ")
string=inp
stk=list("$0")
inp=list(inp+"$")
k=""
check=0
ParseTable=PrettyTable(["Stack", "Input", "Action"])
while(k!="#" and k!="acc"):
  j=int(stk[-1])
  if(inp[0] not in table[j]):
    check=1
    break
  k=table[j][inp[0]]
  el1="".join(stk)
  el2="".join(inp)
  if(k=="acc"):
    el3="Accept"
    flag=1
  elif(k=="#"):
    el3="Error"
    flag=0
  else:
    el3=k
  ParseTable.add_row([el1,el2,el3])
  if(k[0]=="S"):
    s=[inp.pop(0),k[1]]
    stk+=s
  elif(k[0]=="R"):
    num=2*len(rhsAugmented[int(k[1])])
    stk=stk[:-num]
    last=stk[-1]
    ntt=lhsAugmented[int(k[1])]
    stk.append(ntt)
    stk.append(table[int(last)][ntt])
print(ParseTable)
if(flag==0 or check==1):
  print("String ",string,"is not accepted")
else:
  print("String ",string,"is accepted")

Input the string to be parsed: **x=x
+----------+--------+--------+
|  Stack   | Input  | Action |
+----------+--------+--------+
|    $0    | **x=x$ |   S5   |
|   $0*5   | *x=x$  |   S5   |
|  $0*5*5  |  x=x$  |   S4   |
| $0*5*5x4 |  =x$   |   R4   |
| $0*5*5V8 |  =x$   |   R3   |
| $0*5*5E7 |  =x$   |   R5   |
|  $0*5V8  |  =x$   |   R3   |
|  $0*5E7  |  =x$   |   R5   |
|   $0V2   |  =x$   |   S6   |
|  $0V2=6  |   x$   |  S11   |
| $0V2=6x1 |   $    | Accept |
+----------+--------+--------+
String  **x=x is accepted


### **Complete Code to generate parsing table and states in state diagram**

In [ ]:
#Declaring empty lists
productionRules=[] #Production rules   
nonTerminals=[]# non terminals      
rhsAugmented=[]#rhs of augmented rules 
lhsAugmented=[] #lhs of augmented rules 
 
print("Enter production rules, input $ to terminate: ")
 
#Taking first rule as input and adding $ to terminate
a=input()
productionRules.append("G->"+a[0]+"$")
lhsAugmented.append("G")
nonTerminals.append("G")
rhsAugmented.append(a[0]+"$")
#Take rules until "$" is given as an input
while(a!="$"):
  productionRules.append(a)
  a=input()
 
#Generating lhs and rhs for the augmented grammar
for a in productionRules[1:]:
  nonTerminals.append(a.split("->")[0])
  lhsAugmented+=[a.split("->")[0] for m in a.split("->")[1].split("|") ]
  rhsAugmented+=a.split("->")[1].split("|")
 
#Generating rules for the augmented grammar
rules=[]
for i in range(len(lhsAugmented)):
  rules.append(lhsAugmented[i]+"->"+rhsAugmented[i])
 
#Generating terminals of the grammar
terminals=[]
for x in rhsAugmented:
  for char in x:
    if(char not in nonTerminals and char not in terminals):
      terminals.append(char)
 
#Printing rules
print("\nAugmented Grammar: ")
for i in range(len(rules)):
  print(i," ",rules[i])
 
#Functions---------------------------------------------
def first(String):
  if(String[0] not in nonTerminals):
    return String[0]
  firstSet=[]
  for rule in rules:
    if(rule[0]==String[0]):
      firstInRHS=rule.split("->")[1][0]
      if(firstInRHS not in nonTerminals and firstInRHS not in firstSet):
        firstSet.append(firstInRHS)
      elif(firstInRHS!=String[0]):
        tempFirstSet=first(firstInRHS)
        firstSet+=[i for i in tempFirstSet if i not in firstSet]
  return firstSet
 
 
def closure(i,rule,lookahead):
  lst=[[i,rule,lookahead]]
  rhsOfRule=rule.split("->")[1] #t
 
  #Confirming dot isn't in the end of the rule
  if(len(rhsOfRule)!=i):
    for lstRule in lst:
      rhs=lstRule[1].split("->")[1]
      for rule in rules:
        if(rule[0]==rhs[lstRule[0]]):
          lookaheads=first(rhs[i+1:]+lstRule[2])
          for la in lookaheads:
            temp=[0,rule,la]
            if(temp not in lst):
              lst.append(temp)
  return lst
 
 
def symbols(state):
  symb=[]
  for entry in state:
    rhs=entry[1].split("->")[1]
    #Confirming if dot hasn't reached the end of rule
    if(len(rhs)!=entry[0]):
      temp=rhs[entry[0]]
      if(temp not in symb):
        symb.append(temp)
  return symb
 
def goto(state,symbol):
 
  #Checking for the accept state
  if(state[0][2]=="?" and symbol=='$'):
    index=state[0][0]
    if(state[0][1].split("->")[1][index]=="$"):
      return "acc"
 
  newState=[]
  for rule in state:
    rhs=rule[1].split("->")[1]
    if(len(rhs)!=rule[0]):
      if(rhs[rule[0]]==symbol):
        stateForSymbol=closure(rule[0]+1,rule[1],rule[2])
        for tempRule in stateForSymbol:
          if(tempRule not in newState):
            newState.append(tempRule)
        #newState+=stateForSymbol
  return newState 
 
#-----------------------------------------------
#-----------------------------------------------
#Creating a dictionary with set of terminals and non terminals as keys to use in parsing table
columns=terminals+nonTerminals
columns.remove("G")
emptyRow=dict.fromkeys(columns)
for i in emptyRow:
  emptyRow[i]="#"
table=[]
 
#Driver Code along with making entries in table
C=[closure(0,rules[0],"?")]
for state in C:
  #Make a copy of empty row for each state in C
  row=emptyRow.copy()
  symbolsList=symbols(state)
  #For all the symbols in a particular state
 
  for symbol in symbolsList:
   next=goto(state,symbol)
   if(next=="acc"):
     row[symbol]="acc"
   elif(len(next)!=0):
    if(next not in C):
      C.append(next)
    if(symbol not in nonTerminals):
        row[symbol]="S"+str(C.index(next))
    else:
        row[symbol]=str(C.index(next))
  table.append(row)
#---------------------------------------
#-----------------------------------------
#Filling for the reductions
for state in C:
  for entry in state:
    rh=entry[1].split("->")[1]
    if(len(rh)==entry[0]):
      table[C.index(state)][entry[2]]="R"+str(rules.index(entry[1]))
#-------------------------------------------------
 
#Printing parsing table
print("\nParsing Table:")
print("\t","\t".join(columns))
for i in range(len(table)):
  print(i, end="\t")
  for j in table[i]:
    print(table[i][j],end="\t")
  print()
 
#Printing states:
from prettytable import PrettyTable
print("\n\nStates: ")
print("LS stands for look ahead symbols")
for i in range(len(C)):
  print("\nI"+str(i)+":")
  State=PrettyTable(["Rule", "LS"])
  for j in C[i]:
    dotposition=j[0]
    lhs=j[1].split("->")[0]
    rhs=j[1].split("->")[1]
    rhswithdot=rhs[0:dotposition]+"."+rhs[dotposition:]
    rule=lhs+"->"+rhswithdot
    State.add_row([rule,j[2]])
  print(State)

Enter production rules, input $ to terminate: 
S->CC
C->eC|d
$

Augmented Grammar: 
0   G->S$
1   S->CC
2   C->eC
3   C->d

Parsing Table:
	 $	e	d	S	C
0	#	S3	S4	1	2	
1	acc	#	#	#	#	
2	#	S6	S7	#	5	
3	#	S3	S4	#	8	
4	#	R3	R3	#	#	
5	R1	#	#	#	#	
6	#	S6	S7	#	9	
7	R3	#	#	#	#	
8	#	R2	R2	#	#	
9	R2	#	#	#	#	


States: 
LS stands for look ahead symbols

I0:
+--------+----+
|  Rule  | LS |
+--------+----+
| G->.S$ | ?  |
| S->.CC | $  |
| C->.eC | e  |
| C->.eC | d  |
| C->.d  | e  |
| C->.d  | d  |
+--------+----+

I1:
+--------+----+
|  Rule  | LS |
+--------+----+
| G->S.$ | ?  |
+--------+----+

I2:
+--------+----+
|  Rule  | LS |
+--------+----+
| S->C.C | $  |
| C->.eC | $  |
| C->.d  | $  |
+--------+----+

I3:
+--------+----+
|  Rule  | LS |
+--------+----+
| C->e.C | e  |
| C->.eC | e  |
| C->.d  | e  |
| C->e.C | d  |
| C->.eC | d  |
| C->.d  | d  |
+--------+----+

I4:
+-------+----+
|  Rule | LS |
+-------+----+
| C->d. | e  |
| C->d. | d  |
+-------+----+

I5:
+--------+----+
|  Rule  |

### **Parsing string**

In [ ]:
from prettytable import PrettyTable
inp=input("Input the string to be parsed: ")
string=inp
stk=list("$0")
inp=list(inp+"$")
k=""
check=0
ParseTable=PrettyTable(["Stack", "Input", "Action"])
while(k!="#" and k!="acc"):
  j=int(stk[-1])
  if(inp[0] not in table[j]):
    check=1
    break
  k=table[j][inp[0]]
  el1="".join(stk)
  el2="".join(inp)
  if(k=="acc"):
    el3="Accept"
    flag=1
  elif(k=="#"):
    el3="Error"
    flag=0
  else:
    el3=k
  ParseTable.add_row([el1,el2,el3])
  if(k[0]=="S"):
    s=[inp.pop(0),k[1:]]
    stk+=s
  elif(k[0]=="R"):
    num=2*len(rhsAugmented[int(k[1])])
    stk=stk[:-num]
    last=stk[-1]
    ntt=lhsAugmented[int(k[1])]
    stk.append(ntt)
    stk.append(table[int(last)][ntt])
print(ParseTable)
if(flag==0 or check==1):
  print("String",string,"is rejected")
else:
  print("String",string,"is accepted")

Input the string to be parsed: ede
+--------+-------+--------+
| Stack  | Input | Action |
+--------+-------+--------+
|   $0   |  ede$ |   S3   |
|  $0e3  |  de$  |   S4   |
| $0e3d4 |   e$  |   R3   |
| $0e3C8 |   e$  |   R2   |
|  $0C2  |   e$  |   S6   |
| $0C2e6 |   $   | Error  |
+--------+-------+--------+
String ede is rejected


### **More Grammars for testing:**


In [ ]:
"""
S->aAd|bBd|aBe|bAe
A->c
B->c

S->CC
C->eC
C->d

E->T+E|T
T->x

A->(A)|a

Left Recursion:
A->ABd|a
B->Be|b

Left Factoring:
S->aA|aB
A->a 
B->b
"""

In [ ]:
'''
Ambiguous Grammars we tested on:
S->iEtS|iEtSeS|a
E->b

E->E+E|E*E|(E)|i
'''